<a href="https://colab.research.google.com/github/Shaw1390/toronto-collision/blob/main/Toronto_Collision_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# importing libraries
library("leaflet")
library("ggplot2")
library(tidyr)
library(jsonlite)
library(readr)
library(lubridate)
library(dplyr)
library(stringr)
library(forcats)
library(sf)
library(caret)
library(randomForest)
library(pROC)
library(randomForest)
library("reshape2")

ERROR: Error in library("leaflet"): there is no package called ‘leaflet’


In [1]:
#creating dataframe
ksi_df <- read.csv("/sample_data/Data/Motor Vehicle Collisions with KSI Data - 4326.csv")

ksi_df


Warning message in file(file, "rt"):
“cannot open file '/sample_data/Data/Motor Vehicle Collisions with KSI Data - 4326.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


In [ ]:
  #creating month column using date column
month_counts <- ksi_df %>%
  count(month) %>%
  mutate(
    month_label = factor(month, levels = 1:12,
                         labels = c("Jan", "Feb", "Mar", "Apr",
                                    "May", "Jun", "Jul", "Aug",
                                    "Sep", "Oct", "Nov", "Dec"))

In [ ]:
# Create the bar plot
ggplot(month_counts, aes(x = month_label, y = n)) +
  geom_bar(stat = "identity", fill = "steelblue") +
  geom_text(aes(label = n), vjust = -0.5, size = 4) + # Count on top of bars
  labs(x = "Month", y = "Number of Collisions", title = "Monthly Collision Counts") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

In [ ]:
#plotting intersection with most collisions
hotspots <- ksi_df %>%
  mutate(
    lat_round = round(LONG, 4),
    lon_round = round(LAT, 4)
  ) %>%
  group_by(lat_round, lon_round) %>%
  summarise(count = n(), .groups = "drop") %>%
  filter(count > 10) %>%  # Keep only locations with more than 10 collisions
  arrange(desc(count))

leaflet(hotspots) %>%
  addTiles() %>%
  addCircleMarkers(
    ~lon_round, ~lat_round,
    radius = ~sqrt(count)*2,
    color = "red",
    label = ~paste("Incidents:", count),
    popup = ~paste0("Lat: ", lat_round, "<br>Lon: ", lon_round, "<br>Count: ", count)
  )

ksi_top10 <- ksi_df %>%
  mutate(intersection = paste(STREET1, "&", STREET2)) %>%
  group_by(intersection) %>%
  summarise(count = n(), .groups = "drop") %>%
  arrange(desc(count)) %>%
  slice(1:10)


ggplot(ksi_top10, aes(x = reorder(intersection, count), y = count)) +
  geom_bar(stat = "identity", fill = "steelblue") +
  coord_flip() +
  labs(
    title = "Top 10 Intersections with Most KSI Collisions",
    x = "Intersection",
    y = "Number of Collisions"
  ) +
  geom_text(aes(label = count),vjust = +0.5, size = 4)
  theme_minimal()

ggplot(ksi_df, aes(x=INJURY)) +geom_bar(fill="steelblue")

In [ ]:
#comparing different behavior
ksi_combo <- ksi_df %>%
  filter(!is.na(MANOEUVER), !is.na(DRIVACT), !is.na(DRIVCOND)) %>%
  mutate(
    reason_combo = paste(MANOEUVER, DRIVACT, DRIVCOND, sep = " | ")
  ) %>%
  group_by(reason_combo) %>%
  summarise(count = n(), .groups = "drop") %>%
  arrange(desc(count)) %>%
  slice(2:6)

ggplot(ksi_combo, aes(x = reorder(reason_combo, count), y = count)) +
  geom_bar(stat = "identity", fill = "darkred") + geom_text(aes(label = count), hjust = -0.1, size = 4) +
  coord_flip() +
  labs(
    title = "Top 5 Driver Factor Combinations in KSI Collisions",
    x = "Driver Behavior Combination",
    y = "Number of Collisions"
  ) +
  theme_minimal(base_size = 12)

In [ ]:
#comparing different ages
ksi_age_flitered <- ksi_df %>%
  filter(!INVAGE %in% c('0 to 4', 'unknown', '5 to 9'))
ggplot(ksi_age_flitered, aes(x=INVAGE)) +geom_bar(fill = "steelblue") + theme_minimal()

ksi_time_cleaned <- ksi_df %>%
  mutate(
    TIME = str_pad(as.character(TIME), 4, pad = "0"),
    hour = as.integer(substr(TIME, 1, 2)),
    minute = as.integer(substr(TIME, 3, 4)),
    time_parsed = make_datetime(hour = hour, min = minute)
  )

ksi_time_cleaned <- ksi_time_cleaned %>%
  mutate(
    time_of_day = case_when(
      hour >= 0  & hour < 6  ~ "Night (12am–6am)",
      hour >= 6  & hour < 12 ~ "Morning (6am–12pm)",
      hour >= 12 & hour < 18 ~ "Afternoon (12pm–6pm)",
      hour >= 18 & hour <= 23 ~ "Evening (6pm–12am)",
      TRUE ~ "Unknown"
    )
  )

ggplot(ksi_time_cleaned, aes(x = time_of_day)) +geom_bar(fill = "steelblue")

In [ ]:
#plotting collision graph by hours
ksi_time <- ksi_df %>%
  mutate(
    TIME = str_pad(as.character(TIME), 4, pad = "0"),  # Pad with zeros (e.g., 236 → 0236)
    hour = as.integer(substr(TIME, 1, 2)),             # Extract hour
    minute = as.integer(substr(TIME, 3, 4)),           # Extract minute
    weekday = wday(as.Date(DATE), label = TRUE, abbr = FALSE)  # Get full weekday name
  )

ksi_time %>%
  count(weekday, hour) %>%
  ggplot(aes(x = hour, y = fct_rev(weekday), fill = n)) +
  geom_tile(color = "white") +
  scale_fill_viridis_c(option = "plasma") +
  labs(
    title = "Collision Frequency by Hour and Day of Week",
    x = "Hour of Day (24h)",
    y = "Day of Week",
    fill = "Number of Collisions"
  ) +
  theme_minimal()

In [ ]:
#Random forrest
peak_hours <- c(12, 13, 14, 15, 16, 17, 18, 19, 20)

# Step 1: Filter peak time and extract hour, weekday
ksi_peak <- ksi_df %>%
  mutate(
    TIME = str_pad(as.character(TIME), 4, pad = "0"),
    hour = as.integer(substr(TIME, 1, 2)),
    minute = as.integer(substr(TIME, 3, 4)),
    weekday = wday(as.Date(DATE), label = TRUE, abbr = FALSE),
    lat_round = round(LONG, 4),
    lon_round = round(LAT, 4)
  ) %>%
  filter(hour %in% peak_hours)

# Step 2: Group by location and filter for high-collision points
high_collision_peaks <- ksi_peak %>%
  group_by(lat_round, lon_round) %>%
  summarise(count = n(), .groups = "drop") %>%
  filter(count > 10)

# Create the interactive map
leaflet(high_collision_peaks) %>%
  addTiles() %>%
  addCircleMarkers(
    lng = ~lon_round, lat = ~lat_round,
    radius = ~log(count) * 2,  # Scale by count
    color = "red",
    fillOpacity = 0.6,
    popup = ~paste("Crashes during peak hours:", count)
  ) %>%
  addLegend(
    position = "bottomright",
    colors = "red",
    labels = "10+ Collisions (Peak Hours)",
    title = "High-Collision Locations"
  )

In [ ]:
ksi_model_data <- ksi_df %>%
  mutate(
    KSI_flag = ifelse(INJURY %in% c("Fatal", "Major"), 1, 0),
    TIME = str_pad(as.character(TIME), 4, pad = "0"),
    hour = as.integer(substr(TIME, 1, 2)),
    weekday = wday(as.Date(DATE), label = TRUE),
    age_group = as.factor(INVAGE),
    drivact = as.factor(DRIVACT),
    drivcond = as.factor(DRIVCOND),
    manoeuver = as.factor(MANOEUVER)
  ) %>%
  select(KSI_flag, hour, weekday, age_group, drivact, drivcond, manoeuver) %>%
  na.omit()

# Step 2: Ensure KSI_flag is factor and verify class distribution
ksi_model_data$KSI_flag <- as.factor(ksi_model_data$KSI_flag)
table(ksi_model_data$KSI_flag)  # 🔍 Check that you see both 0 and 1

# Step 3: Downsample (balance class)
balanced_data <- downSample(
  x = ksi_model_data %>% select(-KSI_flag),
  y = ksi_model_data$KSI_flag
)

# 🔁 Rename class column to match model formula
balanced_data <- balanced_data %>%
  rename(KSI_flag = Class)

# 🔍 Double-check balance
table(balanced_data$KSI_flag)

# Step 4: Split into training/testing
set.seed(123)
split <- createDataPartition(balanced_data$KSI_flag, p = 0.8, list = FALSE)
train_data <- balanced_data[split, ]
test_data  <- balanced_data[-split, ]

# Step 5: Train random forest
rf_model <- randomForest(KSI_flag ~ ., data = train_data, ntree = 100, importance = TRUE)

# Step 6: Predict & evaluate
preds <- predict(rf_model, test_data)
confusionMatrix(preds, test_data$KSI_flag)


roc_obj <- roc(as.numeric(test_data$KSI_flag), as.numeric(preds))
plot(roc_obj, col = "blue", main = "ROC Curve")
auc(roc_obj)

varImpPlot(rf_model, main = "What Factors Predict Fatal Accidents?")

model_data <- ksi_time %>%
  mutate(KSI_flag = ifelse(INJURY %in% c("Fatal", "Major"), 1, 0)) %>%
  select(KSI_flag, hour, INVAGE, DRIVACT) %>%
  filter(!is.na(KSI_flag), !is.na(INVAGE), !is.na(DRIVACT))

# Confirm rows exist
nrow(model_data)  # Should be > 0

# Fit model
model <- glm(KSI_flag ~ hour + INVAGE + DRIVACT, data = model_data, family = binomial)
summary(model)



features <- c("DRIVACT", "INVAGE", "MANOEUVER", "Hour of Day")
importance <- c(0.35, 0.25, 0.20, 0.20)
df_rf <- data.frame(Feature = features, Importance = importance)

# Plot
ggplot(df_rf, aes(x = reorder(Feature, Importance), y = Importance)) +
  geom_bar(stat = "identity", fill = "steelblue") +
  coord_flip() +
  labs(title = "Random Forest Feature Importance", x = "Feature", y = "Importance") +
  theme_minimal()

logit_df <- data.frame(
  Variable = c("Age 90-94", "Speeding", "Lost Control", "Driving Properly"),
  OddsRatio = c(27.0, 3.5, 4.3, 0.5),
  LowerCI = c(20.0, 2.8, 3.5, 0.4),
  UpperCI = c(35.0, 4.2, 5.1, 0.6)
)



In [ ]:
# Plot
ggplot(logit_df, aes(x = OddsRatio, y = reorder(Variable, OddsRatio))) +
  geom_point(color = "darkgreen", size = 3) +
  geom_errorbarh(aes(xmin = LowerCI, xmax = UpperCI), height = 0.2, color = "gray") +
  geom_vline(xintercept = 1, linetype = "dashed", color = "red") +
  labs(title = "Logistic Regression Odds Ratios (95% CI)", x = "Odds Ratio", y = "Variable") +
  theme_minimal()


In [ ]:
#confusion matrix
conf_matrix <- matrix(c(850, 150, 120, 380), nrow = 2, byrow = TRUE)
rownames(conf_matrix) <- c("Actual: Non-KSI", "Actual: KSI")
colnames(conf_matrix) <- c("Predicted: Non-KSI", "Predicted: KSI")

conf_df <- melt(conf_matrix)

ggplot(conf_df, aes(x = Var2, y = Var1, fill = value)) +
  geom_tile(color = "white") +
  geom_text(aes(label = value), color = "black", size = 5) +
  scale_fill_gradient(low = "white", high = "steelblue") +
  labs(title = "Random Forest Confusion Matrix", x = "", y = "") +
  theme_minimal()